# Rogue Trader enemies

## Introduction

### Index

### Libraries

In [1]:
import pandas as pd
import numpy as np
import json
import os
import re
from bs4 import BeautifulSoup
import jsonpath_ng as jp
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 300)
import warnings
import openpyxl
warnings.filterwarnings('ignore')
import flatten_json as fj

### Functions

In [2]:
# Fuction to show missed data in a table
def missing_values_tab(df):
    #Count missed values
    mis_val = df.isnull().sum()
    #Count share of missed values of all data 
    mis_val_percent = round(100 * df.isnull().sum() / len(df),2)
    
    mv_table = pd.concat([mis_val, mis_val_percent], axis=1)
    
    mv_table = mv_table.rename(columns = {0 : 'missed_values', 1 : '%_of_all'})
    
    mv_table['data_type'] = df.dtypes
    mv_table = mv_table[mv_table.iloc[:,1]!=0].sort_values(by='missed_values',ascending=False)
    print ("Dataframe contains " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " strings.\n")
    print("It has  " + str(mv_table.shape[0]) +" columns with missed values")    
    return display(mv_table) 

In [3]:
def camel_case_split(str): 
    return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)

In [4]:
def column_name_formatter(col):
    """used for delete prefixes 'Data_m_' in columns' names
    """ 
    if 'Data' in col:
        parts = col.split('_')
        new_name = parts[-1]
        return new_name
    else:
        return col

In [5]:
# Функция для очистки строк от префикса '!bp_'
def clear_rows(row):
        
    if re.match('^\!bp_\w+', str(row)):
        tmp = row.split('_')
        new_value = tmp[1]        
        return new_value
    else:
        return row

In [6]:
#Function to extract json into dataframes
def create_df(filelist):
    recipes_df = pd.DataFrame()
    dfs_list = []
    for file in filelist:
        with open(file) as user_file:
            file_contents = user_file.read()
            data = json.loads(file_contents)
            dic = (fj.flatten(data))
            df = pd.DataFrame.from_dict(dic, orient='index')
            dfs_list.append(df)
    for df in dfs_list:
        recipes_df = recipes_df.join(df, how = 'outer',lsuffix = 'x_',rsuffix = 'y_')
    #assign new column names
    column_numbers = list(range(len(recipes_df.columns)))
    recipes_df.columns = column_numbers
    #fill Nans
    #recipes_df = recipes_df.fillna(0)
    #remove prefix '!bp_'
    for col in recipes_df.columns:
        recipes_df[col] = recipes_df[col].apply(clear_rows)  
    #transpose dataframe
    recipes_df = recipes_df.T
    #-------------------------------------
    cols_to_drop = []
    for col in recipes_df.columns:
        if 'VisualParameters' in col or 'FXSettings' in col: #drop columns with visual parameters
            cols_to_drop.append(col)
        elif 'Data_Comment' in col or 'PrototypeLink' in col:  #drop columns with dev comments
            cols_to_drop.append(col)
        elif 'Meta_ShadowDeleted' in col: #drop trash column
            cols_to_drop.append(col)
        elif 'JsonPath' in col or 'PropertyPath' in col or 'OwnerString' in col: #drop columns with fileways
            cols_to_drop.append(col)
        elif 'UI' in col or 'Sound' in col or 'Icon' in col: #drop columns with sound, ui properties
            cols_to_drop.append(col)
    recipes_df = recipes_df.drop(cols_to_drop, axis = 1)
    # drop columns with empty lists dicts or nans in all cells
    recipes_df = recipes_df.loc[:,~recipes_df.where(recipes_df.astype(bool)).isna().all(axis = 0)]

    #add column Filepath
    recipes_df['Filepath'] = filelist
    return recipes_df

In [7]:
# Function to get rid of characters between curly brackets
""" Sometimes not working correctly
def format_description(row):
    if '{' in row:
        row = re.sub(r'\s*{.*}\s*','',row)
    return row
"""
def format_description(row):   
    openining_braces = '\{'
    closing_braces = '\}'
    text_between = '.*?'
    x = re.sub(f'[{openining_braces}]{text_between}[{closing_braces}]','',row)
    #get rid of html tags
    x = re.sub('<[^>]*>',"",x)
    return x

In [8]:
#Function to find columns with string keys
def match_pattern(row):
        if re.match('^\w{8}-\w{4}-\w{4}-\w{4}-\w{12}$', str(row)):
            return True   
        else:
            return False
   
        
match_pattern('ca24785c24384d142a6c22d7585bb3a6, BlueprintWeaponEnchantment')
"""
#finding columns with string keys
stringkey_cols = []
for col in weapon_enchants_df.columns:    
        a = weapon_enchants_df[col].apply(match_pattern).any()
        if a:            
            stringkey_cols.append(col)
print(stringkey_cols)   
    """

'\n#finding columns with string keys\nstringkey_cols = []\nfor col in weapon_enchants_df.columns:    \n        a = weapon_enchants_df[col].apply(match_pattern).any()\n        if a:            \n            stringkey_cols.append(col)\nprint(stringkey_cols)   \n    '

In [9]:
#Function to find columns with asset ids
def find_assetIds_cols(row):
        if re.match('^[a-zA-Z0-9]{32}$', str(row)):
            return True   
        else:
            return False      

"""
#finding columns with asset ids
assetIds_cols = []
for col in weapon_enchants_df.columns:    
        a = weapon_enchants_df[col].apply(find_assetIds_cols).any()
        if a:            
            assetIds_cols.append(col)
print(assetIds_cols)   
    """

'\n#finding columns with asset ids\nassetIds_cols = []\nfor col in weapon_enchants_df.columns:    \n        a = weapon_enchants_df[col].apply(find_assetIds_cols).any()\n        if a:            \n            assetIds_cols.append(col)\nprint(assetIds_cols)   \n    '

In [10]:
#Function to find columns with context actions, containing $ signs
def find_context_columns(row):
        if re.match('^\$[A-Za-z0-9]+\$\w{8}-\w{4}-\w{4}-\w{4}-\w{12}$', str(row)):
            return True   
        else:
            return False
"""
#find columns with rows containing $$
context_cols = []
for col in weapon_enchants_df.columns:    
        a = weapon_enchants_df[col].apply(find_context_columns).any()
        if a:            
            context_cols.append(col)
print(context_cols)   
    """

'\n#find columns with rows containing $$\ncontext_cols = []\nfor col in weapon_enchants_df.columns:    \n        a = weapon_enchants_df[col].apply(find_context_columns).any()\n        if a:            \n            context_cols.append(col)\nprint(context_cols)   \n    '

In [11]:
def find_item_type(row):
    """Function to return item type from the column "Data_$type"
    """
    tmp = str(row).split(',')
    data_type = tmp[1]    
    parts = re.split('(?<=[a-z])(?=[A-Z])', data_type)
    item_type = parts[-1]   
    return item_type

In [12]:
def create_text_columns(df, index_df, columns_in, column_out, format_text=False):
    """ Function to create text columns with names and descriptions got from 
        stringkeys in index file in the form of dict
        """    
    # list of columns with string keys of items' names
    cols_names = columns_in
    #created column
    column_out = str(column_out )
    index_df = index_df # index_dict object got from parsing json file with text information
    index_dict = dict(zip(index_df['index'],index_df['text'])) #dict for mapping text information between two tables    
    
    #_____________________________Get items names and descriptions__________________________ 
    
    for col in cols_names:
        list_of_ids = df[col].unique() # list of stringkeys
        #filter out empty elements
        list_of_ids = list(filter(None,list_of_ids))
        for id in list_of_ids:
            try: #if stringkey is in index file return text
                df.loc[df[col]==id, column_out] = df[col].map(index_dict)
            except: #if not return empty row
                df.loc[df[col]==id, column_out] = ''          
    
         # fill missed values with empty row       
    df[column_out] = df[column_out].fillna('')
    # format description to get rid of {}
    if format_text == True:
        df[column_out] = df[column_out].apply(format_description)

   
    return df

In [13]:
def find_property_cols(row):
    #if  re.match('^Data_Components_[0-9]_\$type$',row) or re.match('^Data_Components_[0-9]_m_(?:Fact|Facts)(?:$|_[0-9]$)',row):   
    if re.match('^Data_Components_[0-9]_(?:\$type$|m_(?:Fact|Facts)(?:$|_[0-9]$))',str(row)):
        return True
    else:
        return False

"""
#find columns with rows containing property values
property_cols = []
for col in equip_items_df.columns:    
        a = equip_items_df[col].apply(find_property_cols).any()
        if a:            
            property_cols.append(col)
print(property_cols)   
    """    

'\n#find columns with rows containing property values\nproperty_cols = []\nfor col in equip_items_df.columns:    \n        a = equip_items_df[col].apply(find_property_cols).any()\n        if a:            \n            property_cols.append(col)\nprint(property_cols)   \n    '

In [14]:
def find_properties(row):
    #if  re.match('^Data_Components_[0-9]_\$type$',row) or re.match('^Data_Components_[0-9]_m_(?:Fact|Facts)(?:$|_[0-9]$)',row):   
    if re.match('^Data_Components_[0-9]_(?:\$type$|m_[A-Za-z]+(?:$|_[0-9]$))',str(row)):
        return True
    else:
        return False
def select_columns(df, keywords):
    """function to select columns to keep from a dataframe and return 4 lists of
        stringkey columns, assetids columns, misc columns and combined list of columns
    Arguments: df - dataframe to choose columns from, 
                keywords - list of keywords or names of columns to select for misc list    
    """
    selected_cols = [] # list of needed columns
    assetIds_cols = [] #list of columns with asset ids
    stringkey_cols = [] # list of columns with string keyes
    misc_cols = [] # list of misc columns
    property_cols = [] # list of property columns
    intro_cols = []   # first columns: AssetId and Data_$type in any table

    def find_ids(col):
        if re.match('^[a-z0-9]{32}(?:$|[, A-Za-z])', str(col)):
            return True
        else:
            return False      

    #find columns with asset ids
    for col in df.columns:
        if col !='AssetId':
            a = df[col].apply(find_assetIds_cols).any()
            if a:            
                assetIds_cols.append(col)
        else:
            intro_cols.append(col)

    #find columns with string keys
    for col in df.columns:    
        a = df[col].apply(match_pattern).any()
        if a:            
            stringkey_cols.append(col)
    
    #find columns with rows containing property values

    for col in df.columns: 
        # if name of the column is according to the pattern   
        if  re.match('^Data_Components_[0-9]+_(?:\$type$|m_[A-Za-z]+(?:$|_[0-9]+$))',str(col)):
            a = df[col].apply(find_ids).any()
            if a:                        
        #and its values are assetIds keys  add to property list                 
                property_cols.append(col) 
      
    # find miscellanious columns
    if len(keywords)>0:
        keywords = keywords
        for keyword in keywords:
            for col in df.columns:
                if keyword in col:
                    misc_cols.append(col)
    #else:
        #misc_cols = [d for d in df.columns if d not in assetIds_cols if d not in stringkey_cols if d not in property_cols]
    #update assetIds_cols
    assetIds_cols = [d for d in assetIds_cols if d not in property_cols]
    #Must- have columns, 'AssetId' and 'Data_$type'
    intro_cols.append(df.columns[1]) #add Data_$type
    #create combined list            
    selected_cols = intro_cols + assetIds_cols+ property_cols + stringkey_cols + misc_cols
      
    return selected_cols, assetIds_cols, stringkey_cols, property_cols, misc_cols

In [15]:
def find_property_cols(row):
    #if  re.match('^Data_Components_[0-9]_\$type$',row) or re.match('^Data_Components_[0-9]_m_(?:Fact|Facts)(?:$|_[0-9]$)',row):   
    if re.match('^Data_Components_[0-9]_(?:\$type$|m_(?:Fact|Facts)(?:$|_[0-9]$))',str(row)):
        return True
    else:
        return False

"""
#find columns with rows containing property values
property_cols = []
for col in equip_items_df.columns:    
        a = equip_items_df[col].apply(find_property_cols).any()
        if a:            
            property_cols.append(col)
print(property_cols)   
    """    
       

'\n#find columns with rows containing property values\nproperty_cols = []\nfor col in equip_items_df.columns:    \n        a = equip_items_df[col].apply(find_property_cols).any()\n        if a:            \n            property_cols.append(col)\nprint(property_cols)   \n    '

In [16]:
def get_text_info(df, map_df, index_df, columns_in, format_text=False):
    """ Function to create text columns with names and descriptions. Works on columns with asset ids
        and get filenames of those asset ids in asset_df and then look into files to extract strinkeys
        """
    # list of columns with string keys of items' names
    cols_names = columns_in
    #created column        
    # map_df - a dataframe for mapping    
    assets_dict = dict(zip(map_df.iloc[:,0],map_df.iloc[:,1])) #dict for mapping text information between two tables
    #reference to dataframe go from localization file    
    def find_stringkeys(row):    
        try:
            row = row.strip()
            with open(row) as asset_file:           
                file_contents = asset_file.read()
                data = json.loads(file_contents)
                try:
                    if data['Data']['m_DisplayName']['Shared'] is not None:
                        key_list = [data['Data']['m_DisplayName']['m_Key'], data['Data']['m_DisplayName']['Shared']['stringkey']]
                        for key in key_list:                     
                            a = match_pattern(key) 
                            if a:                 
                                return key
                    else:
                        key = data['Data']['m_DisplayName']['m_Key']
                        return key
                except Exception as err:
                    print(f'Unexpected {err=}, {type(err)=}',row)
                #print(data['Data']['m_DisplayName']['Shared'])
                
        except:
            return ''    
    
    #_____________________________Get items names and descriptions__________________________ 
    temp_list = []
    columns_out = []
    for col in cols_names:                
                df[col] = df[col].str.strip()         
                filenames_series = df[col].map(assets_dict)
                df['{}_stringkey'.format(col)]  = filenames_series.apply(find_stringkeys)
                temp_list.append('{}_stringkey'.format(col))
                text_column_name = '{}_text'.format(col)
                columns_out.append(text_column_name)   
    for i in range(len(columns_out)):
         col_in = [temp_list[i]]
         df = create_text_columns(df, index_df, col_in, columns_out[i], format_text = True)
    
    df['All_Items'] =df[columns_out].agg(sum)  
    return df, columns_out

In [17]:
def find_item_columns(df):
    #Function to find columns with names like Data_Items_0_m_Item
    item_cols = []
    for col in df.columns:
        try:
            if re.match('^Data_Items_[0-9]+_m_Item$',str(col)):
                item_cols.append(col)
        except Exception as err:
                    print(f'Unexpected {err=}, {type(err)=}',col)
            
    return item_cols

In [18]:
def drop_trash_columns (df):
    df = df.loc[:,~df.where(df.astype(bool)).isna().all(axis = 0)]
    return df

In [19]:
def create_df_item_location(df):
    """"Function to zip item ids and area locations from goods_df into separate dataframe"""
    def find_item_columns(df):
    #Function to find columns with names like Data_Items_0_m_Item
        item_cols = []
        for col in df.columns:
            try:
                if re.match('^Data_Items_[0-9]+_m_Item$',str(col)):
                    item_cols.append(col)
            except Exception as err:
                    print(f'Unexpected {err=}, {type(err)=}',col)
            
        return item_cols
    
    item_cols = find_item_columns(df)
    item_areas_df = pd.DataFrame(columns = ['Item_Id','Area_Name', 'Area_Name_Ru'])

    for col in df[item_cols]:
        temp_df = df[[col, 'Area_Name', 'Area_Name_Ru']]
        temp_df = temp_df.rename(columns = {col:'Item_Id'})  
        item_areas_df = pd.concat([item_areas_df,temp_df], axis = 0)
    item_areas_df = item_areas_df.query('Item_Id!=""').reset_index(drop = True)
    return item_areas_df

In [20]:
def get_faction(filename):
    """Function to extract faction names from filepathes"""
    head, tail = os.path.split(filename)
    temp = tail.split('_')
    return temp[0]

### Folders

In [21]:
#folder with the gameobjects' parameters
work_folder = r'e:\Steam\RogueTrader\WorkFolder\Blueprints'
# file with the english localization
index_file = r'e:\Steam\RogueTrader\Localization\enGB.json'
#file with the russian localization
index_file_ru = r'e:\Steam\RogueTrader\Localization\ruRU.json'
save_folder = r'e:\Steam\RogueTrader\SaveFolder'
load_folder = r'e:\Steam\RogueTrader\LoadFolder'
# units and faction folders
folder_units = r'e:\Steam\RogueTrader\WorkFolder\Blueprints\Units'
folder_factions = r'e:\Steam\RogueTrader\WorkFolder\Blueprints\Factions'

### Files

#### Table of gamefiles ids

In [22]:
assets_df = pd.read_csv(save_folder + r'\asset_ids.csv')
display(assets_df.head())

,AssetId,File
0,23068e0962f9449d9d8e3dea732a5958,e:\Steam\RogueTrader\WorkFolder\Blueprints\CameraRecorderData.jbp
1,d48e03729a253814ebb26d21b4e6a0b3,e:\Steam\RogueTrader\WorkFolder\Blueprints\Player Character.jbp
2,5ebfb51da6304029b71e89fd45619ed3,e:\Steam\RogueTrader\WorkFolder\Blueprints\Player CharacterFemale_VisualSettings.jbp
3,bcf25b9f55e2465a910309fb5aab117c,e:\Steam\RogueTrader\WorkFolder\Blueprints\Player Character_VisualSettings.jbp
4,ddb5c3520fe94966acb57592484fd542,e:\Steam\RogueTrader\WorkFolder\Blueprints\Achievements\PentakillAchievement_TechnicalCounter.jbp


#### Table english localization

In [23]:
#Search strings in index_file
index_df = pd.read_json(index_file).reset_index()
display(index_df.head())

,index,strings
0,000393b6-487b-4e13-a1af-4702957a0ee8,"{'Offset': 0, 'Text': 'Sword of Faith — Area Attack'}"
1,0004ce39-6474-47b1-a65f-85365b83a5b1,"{'Offset': 0, 'Text': 'The {0} archetype has been mastered!'}"
2,0004e905-8ef5-420e-accc-e8dc056caf7e,"{'Offset': 0, 'Text': 'Plasma Ring'}"
3,00074f46-4e59-4dcd-b118-d535a7673eba,"{'Offset': 0, 'Text': '{n}Cassia offers you a reticent smile.{/n} ""I foresaw this answer within the golden sparks of your thoughts.""'}"
4,00092c30-751e-4fc9-a549-a7a717893fa7,"{'Offset': 0, 'Text': 'Steady Hand'}"


In [24]:
#unfold dict in the column "strings"
text = pd.json_normalize(index_df['strings'])
index_df = pd.concat([index_df,text['Text']],axis =1).drop('strings',axis = 1).rename(columns = {'Text':'text'})
display(index_df)

,index,text
0,000393b6-487b-4e13-a1af-4702957a0ee8,Sword of Faith — Area Attack
1,0004ce39-6474-47b1-a65f-85365b83a5b1,The {0} archetype has been mastered!
2,0004e905-8ef5-420e-accc-e8dc056caf7e,Plasma Ring
3,00074f46-4e59-4dcd-b118-d535a7673eba,"{n}Cassia offers you a reticent smile.{/n} ""I foresaw this answer within the golden sparks of your thoughts."""
4,00092c30-751e-4fc9-a549-a7a717893fa7,Steady Hand
...,...,...
51543,fff8edb2-e59c-41f3-a604-2de828b7f7bf,"""You wound me, shereen! Jae Heydari always has a better idea. What would you say to making the good folk ahead of us hurry up a little? Just a smidge."" {n}Her eyes dart playfully over the faces of the people around you.{/n}"
51544,fff9c551-5344-4321-b6b7-ed65b71e3dc3,"""Things I saw in my dreams portended that few would survive the disaster. But your words have warmed my heart, mon-keigh. It is a pleasure to know some have survived after all."""
51545,fffbf867-f810-45c8-b386-b7f5edeafe10,Second Wind
51546,fffc736d-7cb3-4578-8092-bda9cd953dae,"A {mf|male|female} mon-keigh. A pampered upper-caste cub who was fed and equipped to the best of what their race can offer. It certainly arrived to the Dark City recently, the city which has not yet managed to leave a mark of perpetual expectation of doom on its flattened snout. The most gloriou..."


### Table of russian localization

In [25]:
#Search strings in index_file
index_df_ru = pd.read_json(index_file_ru).reset_index()
#unfold dict in the column "strings"
text = pd.json_normalize(index_df_ru['strings'])
index_df_ru = pd.concat([index_df_ru,text['Text']],axis =1).drop('strings',axis = 1).rename(columns = {'Text':'text'})
display(index_df_ru.head())

,index,text
0,000393b6-487b-4e13-a1af-4702957a0ee8,Меч веры — атака по области
1,0004ce39-6474-47b1-a65f-85365b83a5b1,Архетип {0} успешно изучен!
2,0004e905-8ef5-420e-accc-e8dc056caf7e,Плазма-кольцо
3,00074f46-4e59-4dcd-b118-d535a7673eba,"{n}Кассия сдержанно улыбается.{/n} ""В золотистых всполохах ваших мыслей я предвидела этот ответ""."
4,00092c30-751e-4fc9-a549-a7a717893fa7,Верная рука


## Work part

### Loot table

#### Folders

In [26]:
folder_units = r'e:\Steam\RogueTrader\WorkFolder\Blueprints\Units'
folder_factions = r'e:\Steam\RogueTrader\WorkFolder\Blueprints\Factions'
folder_loot = r'e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good'

#### Creation of loot table

In [27]:
list_of_goods_files = []

for root, dirs, files in os.walk(folder_loot):        
        for file in files:           
                list_of_goods_files.append(os.path.join(root, file))
print('The number of files in fodler_loot: ', len(list_of_goods_files))
            
               

The number of files in fodler_loot:  236


In [28]:
goods_df = create_df(list_of_goods_files)
display(goods_df.head())

,AssetId,Data_$type,Data_CargoVolumeAmount,Data_ContainerName,Data_Identify,Data_Items_0_Count,Data_Items_0_m_Item,Data_Items_1_Count,Data_Items_1_m_Item,Data_Items_2_Count,Data_Items_2_m_Item,Data_Items_3_Count,Data_Items_3_m_Item,Data_Items_4_Count,Data_Items_4_m_Item,Data_Items_5_Count,Data_Items_5_m_Item,Data_Items_6_Count,Data_Items_6_m_Item,Data_Items_7_Count,Data_Items_7_m_Item,Data_Items_8_Count,Data_Items_8_m_Item,Data_Items_9_Count,Data_Items_9_m_Item,Data_Setting,Data_Type,Data_m_Area,Filepath
0,919118b9d72840dabef59fb9619a8b20,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,AmourCourtoisBook,False,1,2841782e80bc48b1927677b3a88e392e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AdeptusMechanicus,Good,66b9351d457f4363afebcbca96a506da,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_AmourCourtoisBook.jbp
1,136d2833bd834cdab8cb0e8b65ea1bdc,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,GoodLootHidden,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AdeptusMechanicus,Good,54b22e52a6da46eaa8c0f1fe2049d040,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_GoodLootHidden.jbp
2,0fe886222a234ad8abd7022a2d6ef04f,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,Loot4,False,1,199b3d86b5214c0c8fe483806f3122ce,3,edf1ca959e0f40f4bf5bb64586f816b9,1,04292c838432460396cfce58af861338,1,e29803f0fb444c90a153a2563cf30ae7,1,e239a9c0eaa84444a12b5aff054a0085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AdeptusMechanicus,Good,5abaf24cde1f4674a658da370195f537,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_Loot4.jbp
3,a613e4a00a66493a82eb9609ecb8cc61,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (3),False,1,f123c305357a44f58b1d6449ded7cda7,1,8d9ec005c04a4984ad6032b3f74d84b1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_LootGood (3).jbp
4,aed888571d4c4e69b5e063e36ddddfb5,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (5),False,1,60989b213e8547799e6c17ed9b0238c0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_LootGood (5).jbp


In [29]:
keywords = ['Filepath']
selected_cols, assetIds_cols, stringkey_cols, property_cols, misc_cols = select_columns(goods_df, keywords)

In [30]:
# Fill missing values with empty strings
goods_df = goods_df.fillna("")

**Notes**:  
```Data_m_Area``` contains ids of area files in  the folder```e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Areas\```.  
The scene name stored in keys:  ```['Data']['m_DynamicScene']['m_SceneName']```.    

In [31]:
file = r'e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Areas\Prologue\VoidshipOfficerDeck_AfterPrologue\VoidshipOfficerDeck_AfterPrologue.jbp'
with open(file) as user_file:
            file_contents = user_file.read()
            data = json.loads(file_contents)
data['Data']['m_DynamicScene']['m_SceneName']
df = pd.DataFrame({'AssetId':data['AssetId'],'Area':data['Data']['m_DynamicScene']['m_SceneName']}, index = [0])
df
    

,AssetId,Area
0,66b9351d457f4363afebcbca96a506da,VoidshipOfficerDeck_AfterPrologue_Mechanics


#### Making of table of area files

In [32]:
folder_areas = r'e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Areas'
list_of_area_files = []

for root, dirs, files in os.walk(folder_areas):        
        for file in files:           
                list_of_area_files.append(os.path.join(root, file))
print('The number of files in folder_areas: ', len(list_of_area_files))

The number of files in folder_areas:  1888


areas_df = pd.DataFrame()
for file in list_of_area_files:
    with open(file) as user_file:
            file_contents = user_file.read()
            data = json.loads(file_contents)
            df = pd.DataFrame({'AssetId':data['AssetId'],'Area':data['Data']['m_DynamicScene']['m_SceneName']}, index = [0])
            areas_df = pd.concat([areas_df,df], axis = 0)
          
    

In [33]:
#areas_data = create_df(list_of_area_files) 

In [34]:
areas_data = pd.read_csv(load_folder + r'\areas_data.csv') # main area data table
display(areas_data.head())
display(areas_data.shape)

AssetId  \
0  9ab47ae1ac8e47a698c35324c0602384   
1  24125fa1f0774c0e98dfa6451d92eac7   
2  c1df2481218f4e12860432bb9d69327c   
3  248fe652a5974504b13940da0a555e2b   
4  b4dd07d99e1f42baa22090cf69f7fffa   

                                                  Data_$type  \
0            6181425e72507c148a67d77624419aec, BlueprintArea   
1        60192bc8449d2d944a99c6c58f48814b, BlueprintAreaPart   
2        60192bc8449d2d944a99c6c58f48814b, BlueprintAreaPart   
3   c542bb267f6d4651af99d4c5b3a0df9a, BlueprintAreaMechanics   
4  168173fb7e2cd844d86be7f426467d07, BlueprintAreaEnterPoint   

  Data_ActivationCondition_Operation  Data_AdditionalExperience  \
0                                NaN                        NaN   
1                                NaN                        NaN   
2                                NaN                        NaN   
3                                NaN                        NaN   
4                                NaN                        NaN   

  Data_Alignment Data_AlsoLoadMechanics_0 Data_AlsoLoadMechanics_1  \
0            NaN                      NaN                      NaN   
1            NaN                      NaN                      NaN   
2            NaN                      NaN                      NaN   
3            NaN                      NaN                      NaN   
4            NaN                      NaN                      NaN   

  Data_AlsoLoadMechanics_10 Data_AlsoLoadMechanics_11  \
0                       NaN                       NaN   
1                       NaN                       NaN   
2                       NaN                       NaN   
3                       NaN                       NaN   
4                       NaN                       NaN   

  Data_AlsoLoadMechanics_2 Data_AlsoLoadMechanics_3 Data_AlsoLoadMechanics_4  \
0                      NaN                      NaN                      NaN   
1                      NaN                      NaN                      NaN   
2                      NaN                      NaN                      NaN   
3                      NaN                      NaN                      NaN   
4                      NaN                      NaN                      NaN   

  Data_AlsoLoadMechanics_5 Data_AlsoLoadMechanics_6 Data_AlsoLoadMechanics_7  \
0                      NaN                      NaN                      NaN   
1                      NaN                      NaN                      NaN   
2                      NaN                      NaN                      NaN   
3                      NaN                      NaN                      NaN   
4                      NaN                      NaN                      NaN   

  Data_AlsoLoadMechanics_8 Data_AlsoLoadMechanics_9  \
0                      NaN                      NaN   
1                      NaN                      NaN   
2                      NaN                      NaN   
3                      NaN                      NaN   
4                      NaN                      NaN   

  Data_AnomaliesResearchProgress_0_Anomaly  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

  Data_AnomaliesResearchProgress_1_Anomaly  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

  Data_AnomaliesResearchProgress_2_Anomaly                         Data_Area  \
0                                      NaN                               NaN   
1                                      NaN                               NaN   
2                                      NaN                               NaN   
3                                      NaN  9ab47ae1ac8e47a698c35324c06023

(1888, 2442)

<span style="color:yellow">Saving areas_data</span>

In [35]:
#Saving file
#areas_data.to_csv(save_folder + r'\areas_data.csv', index = False)

In [36]:
keywords = ['Filepath']
selected_cols, assetIds_cols, stringkey_cols, property_cols, misc_cols = select_columns(areas_data, keywords)

'Data_AreaName_Shared_stringkey', 'Data_AreaName_m_Key'

In [37]:
display(
    areas_data[areas_data['Data_$type']=='6181425e72507c148a67d77624419aec, BlueprintArea']
    [['Data_AreaName_Shared_stringkey','Data_AreaName_m_Key','Filepath']]
    )

,Data_AreaName_Shared_stringkey,Data_AreaName_m_Key,Filepath
0,fc17ef7f-f730-46ff-98c6-62b4a5be4829,NaN,e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Areas\Chapter_1\ArcheotechReactor\ArcheotechReactor.jbp
21,fc17ef7f-f730-46ff-98c6-62b4a5be4829,NaN,e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Areas\Chapter_1\ArcheotechReactor_Outdoor\ArcheotechReactor_Outdoor.jbp
26,f5d6c269-e214-49db-9c3f-59227e0da47c,NaN,e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Areas\Chapter_1\CommandCenter\CommandCenter.jbp
29,a805e7bf-f7d8-467e-93c3-cf4919a1fa5a,NaN,e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Areas\Chapter_1\Jail Planetoid\JailPlanetoid.jbp
49,NaN,NaN,e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Areas\Chapter_1\NavigatorRoom\NavigatorRoom.jbp
...,...,...,...
1860,NaN,NaN,e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Areas\Test_Areas\Test_Areas_CinemachineCamera\TestArea_LD_CinemachineCamera.jbp
1863,NaN,NaN,e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Areas\Test_Areas\Tutorial_Scene\Tutorial_Scene.jbp
1866,NaN,NaN,e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Areas\Test_Areas\Tutorial_Test_Polygon\Tutorial_Test_Polygon.jbp
1872,NaN,NaN,e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Areas\Test_Areas\UI_TestPolygon\UI_TestPolygon.jbp


In [38]:
#check what data types are in the table
areas_data['Data_$type'].unique()

array(['6181425e72507c148a67d77624419aec, BlueprintArea',
       '60192bc8449d2d944a99c6c58f48814b, BlueprintAreaPart',
       'c542bb267f6d4651af99d4c5b3a0df9a, BlueprintAreaMechanics',
       '168173fb7e2cd844d86be7f426467d07, BlueprintAreaEnterPoint',
       '700954073a6aeac449fc2fe45dee75d5, BlueprintAreaPreset',
       '251b1bf89b36e384d8e452ce50ea1d8b, BlueprintAreaTransition',
       'cb208b98ceacca84baee15dba53b1979, BlueprintFeature',
       '618a7e0d54149064ab3ffa5d9057362c, BlueprintBuff',
       'b7d695389bff6604ca37f190a61f05b5, BlueprintDynamicMapObject',
       '5e67da75e2474764a5fde3375352ac8f, BlueprintItemsStash',
       '61e2c1e69c5ae5946b86db4b7e880eb9, BlueprintSpaceCombatArea',
       '585edc54e62446a58844f30e5d2f1492, BlueprintSpawnersList',
       '8da007f57f974091916d05cbcc28116a, BlueprintUnitsList',
       '086ea1e7680449d5ab2a7006667f71c8, BlueprintMeteor',
       '751a76a1090f416bbec1990d90844579, BlueprintMeteorStream',
       'd03eea79843045d18c110d9b79cd

Creating a slice from the main area table containing important information such as area names and keys:

In [39]:
# slice from the main table
areas_df = (areas_data[
    (areas_data['Data_$type']=="6181425e72507c148a67d77624419aec, BlueprintArea")]\
[['AssetId', 'Data_m_DynamicScene_m_SceneName','Data_AreaName_Shared_stringkey', 'Data_AreaName_m_Key']])
display(areas_df.head())
# check for duplicate rows
print(areas_df.duplicated().sum())
print(areas_df.shape[0])

,AssetId,Data_m_DynamicScene_m_SceneName,Data_AreaName_Shared_stringkey,Data_AreaName_m_Key
0,9ab47ae1ac8e47a698c35324c0602384,ArcheotechReactor_Mechanics,fc17ef7f-f730-46ff-98c6-62b4a5be4829,NaN
21,aca223699515442992b1a43ca4df100b,ArcheotechReactor_Outdoor_Mechanics,fc17ef7f-f730-46ff-98c6-62b4a5be4829,NaN
26,6e0f19e95a8a4278b169cd00deb53e3b,CommandCenter_Mechanics,f5d6c269-e214-49db-9c3f-59227e0da47c,NaN
29,8250d8a565c84dcdba6bf3064211cdf9,JailPlanetoid_Mechanics,a805e7bf-f7d8-467e-93c3-cf4919a1fa5a,NaN
49,9e1a8bb634ff4edc90c5c756579bbb11,NavigatorRoom_Mechanics,NaN,NaN


0
131


In [40]:
# How many rows have stringkeys in both columns
display(areas_df[(areas_df['Data_AreaName_Shared_stringkey'].notna()) 
                 & (areas_df['Data_AreaName_m_Key'].notna())].shape[0]  
                 )
#how many rows have nans in the column:
display(areas_df[(areas_df['Data_AreaName_Shared_stringkey'].notna())].shape[0] 
                 )
#how many rows have nans in other column:
display(areas_df[(areas_df['Data_AreaName_m_Key'].notna())].shape[0] 
                 )


0

76

25

In [41]:
# Fill nans with empty rows
areas_df = areas_df.fillna('')


Create columns with area names: as both stringkey columns dont have common values, only one column with area names will be created:

In [42]:
# Create column for the enlish version:
columns = ['Data_AreaName_Shared_stringkey', 'Data_AreaName_m_Key']
areas_df = create_text_columns(areas_df, index_df, columns, 'Area_Name')

#create column with area names in the russian language
areas_df = create_text_columns(areas_df, index_df_ru, columns, 'Area_Name_Ru')
display(areas_df.head())

,AssetId,Data_m_DynamicScene_m_SceneName,Data_AreaName_Shared_stringkey,Data_AreaName_m_Key,Area_Name,Area_Name_Ru
0,9ab47ae1ac8e47a698c35324c0602384,ArcheotechReactor_Mechanics,fc17ef7f-f730-46ff-98c6-62b4a5be4829,,Electrodynamic Cenobium,Электродинамическая обитель
21,aca223699515442992b1a43ca4df100b,ArcheotechReactor_Outdoor_Mechanics,fc17ef7f-f730-46ff-98c6-62b4a5be4829,,Electrodynamic Cenobium,Электродинамическая обитель
26,6e0f19e95a8a4278b169cd00deb53e3b,CommandCenter_Mechanics,f5d6c269-e214-49db-9c3f-59227e0da47c,,Command Centre,Командный центр
29,8250d8a565c84dcdba6bf3064211cdf9,JailPlanetoid_Mechanics,a805e7bf-f7d8-467e-93c3-cf4919a1fa5a,,"Rykadi Philia, the Prison Planetoid","Райкади Филия, планетоид-тюрьма"
49,9e1a8bb634ff4edc90c5c756579bbb11,NavigatorRoom_Mechanics,,,,


In [43]:
areas_df[areas_df['Area_Name']==""].shape[0]

38

In [44]:
areas_df['Area_Name'].unique()

array(['Electrodynamic Cenobium', 'Command Centre',
       'Rykadi Philia, the Prison Planetoid', '', 'Eurac V', 'Starport',
       'The Upperway', 'Voidship Bridge', 'Lower Decks', 'Administratum',
       'Forests of Janus', "Governor Vyatt's Estate",
       "The Martyr's Endurance", 'Gutted Voidship', "Governor's Camp",
       'Plasma Drive Chambers', 'Distress Signal Zone', 'Bottomless Pit',
       'Inquisition Shipwreck', 'Unidentified Ruins', "Recolligers' Plot",
       'Ancient Bunker', 'Footfall Atrium',
       'Footfall Void Dock Alpha-Rho', 'Footfall Shadow Quarters',
       "Cultists' lair", 'Dark Hideout', 'Machine Cathedral',
       'Kiava Gamma Manufactorum', 'Manufactorum of Kiava Gamma',
       'Drifting Voidship', 'Inner World', 'Ice World', 'Hive Sewers',
       'Plundered Void Station', 'Wasteland Wayfarer',
       'Macro-Cannon Chamber', 'Upper Decks', 'Pits of Vheabos VI',
       'Von Valancius Palace', 'Hive Depths', 'Hive World',
       'Ravaged Void Station', 'Ch

#### Add area's name to goods table

In [45]:
goods_df = goods_df.merge(areas_df[['AssetId','Area_Name','Area_Name_Ru']], how = 'left',
                           left_on = ['Data_m_Area'],
                             right_on = ['AssetId'], suffixes = (None, '_y')).drop('AssetId_y', axis = 1)
goods_df.head(5)

,AssetId,Data_$type,Data_CargoVolumeAmount,Data_ContainerName,Data_Identify,Data_Items_0_Count,Data_Items_0_m_Item,Data_Items_1_Count,Data_Items_1_m_Item,Data_Items_2_Count,Data_Items_2_m_Item,Data_Items_3_Count,Data_Items_3_m_Item,Data_Items_4_Count,Data_Items_4_m_Item,Data_Items_5_Count,Data_Items_5_m_Item,Data_Items_6_Count,Data_Items_6_m_Item,Data_Items_7_Count,Data_Items_7_m_Item,Data_Items_8_Count,Data_Items_8_m_Item,Data_Items_9_Count,Data_Items_9_m_Item,Data_Setting,Data_Type,Data_m_Area,Filepath,Area_Name,Area_Name_Ru
0,919118b9d72840dabef59fb9619a8b20,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,AmourCourtoisBook,False,1,2841782e80bc48b1927677b3a88e392e,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,66b9351d457f4363afebcbca96a506da,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_AmourCourtoisBook.jbp,Upper Decks,Верхние палубы
1,136d2833bd834cdab8cb0e8b65ea1bdc,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,GoodLootHidden,False,,,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,54b22e52a6da46eaa8c0f1fe2049d040,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_GoodLootHidden.jbp,Ruined Palace,Разрушенный дворец
2,0fe886222a234ad8abd7022a2d6ef04f,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,Loot4,False,1,199b3d86b5214c0c8fe483806f3122ce,3,edf1ca959e0f40f4bf5bb64586f816b9,1,04292c838432460396cfce58af861338,1,e29803f0fb444c90a153a2563cf30ae7,1,e239a9c0eaa84444a12b5aff054a0085,,,,,,,,,,,AdeptusMechanicus,Good,5abaf24cde1f4674a658da370195f537,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_Loot4.jbp,Outer Spire Halls,Внешние палаты шпиля
3,a613e4a00a66493a82eb9609ecb8cc61,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (3),False,1,f123c305357a44f58b1d6449ded7cda7,1,8d9ec005c04a4984ad6032b3f74d84b1,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_LootGood (3).jbp,Upper Decks,Верхние палубы
4,aed888571d4c4e69b5e063e36ddddfb5,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (5),False,1,60989b213e8547799e6c17ed9b0238c0,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_LootGood (5).jbp,Upper Decks,Верхние палубы


In [46]:
#Check if exists empty rows in Area names
display(goods_df[goods_df['Area_Name'].isna()])

,AssetId,Data_$type,Data_CargoVolumeAmount,Data_ContainerName,Data_Identify,Data_Items_0_Count,Data_Items_0_m_Item,Data_Items_1_Count,Data_Items_1_m_Item,Data_Items_2_Count,Data_Items_2_m_Item,Data_Items_3_Count,Data_Items_3_m_Item,Data_Items_4_Count,Data_Items_4_m_Item,Data_Items_5_Count,Data_Items_5_m_Item,Data_Items_6_Count,Data_Items_6_m_Item,Data_Items_7_Count,Data_Items_7_m_Item,Data_Items_8_Count,Data_Items_8_m_Item,Data_Items_9_Count,Data_Items_9_m_Item,Data_Setting,Data_Type,Data_m_Area,Filepath,Area_Name,Area_Name_Ru
12,940110b2e89c4c74a842114742b75464,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,TestLootObj_loot_object,False,1,44570efd38534b0eb3227fad6598e236,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,cfcfef899cb14d10a6463b7dbfc877ba,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_TestLootObj_loot_object.jbp,NaN,NaN


As it is a test area we can drop it:

In [47]:
goods_df = goods_df.dropna(subset=['Area_Name']).reset_index()

#### Add item names to loot table

**Notes:**  
possible folders:  ..\World\....\Items  
                   ..\Equipment\.... files with '_Item' in its names  
                   ..\Consumables\.... 
**Algorithm of getting loot data**:  
1. Iterate through ```assetids_cols``` of goods_df.
2. iterate over rows of the column: if Nan, loot = "", if there is a 32 chr key:
3. Find in ```assets_df``` file with that key.
4. Find stringkeys columns in that file:  ```Data_m_DisplayName_Shared_stringkey``` or 
5. Provide that strinkey to ```index_df``` to get text_name of item.

                      


In [48]:
#find item columns with names like Data_Items_{number}_m_Item
columns = find_item_columns(goods_df)
goods_df, columns = get_text_info(df = goods_df,
                            map_df = assets_df[['AssetId','File']],
                            index_df = index_df,
                            columns_in = columns)

In [49]:
#Create aggregate column of contained items in area
goods_df['All_Items'] = goods_df[columns].astype(str).agg(lambda x: ', '.join(word for word in x if word.strip()), axis = 1)
#Create aagregate column of item ids
item_cols = find_item_columns(goods_df)
goods_df['All_Item_Ids'] = goods_df[item_cols].astype(str).agg(lambda x: ', '.join(word for word in x if word.strip()), axis = 1)
goods_df.head()

,index,AssetId,Data_$type,Data_CargoVolumeAmount,Data_ContainerName,Data_Identify,Data_Items_0_Count,Data_Items_0_m_Item,Data_Items_1_Count,Data_Items_1_m_Item,Data_Items_2_Count,Data_Items_2_m_Item,Data_Items_3_Count,Data_Items_3_m_Item,Data_Items_4_Count,Data_Items_4_m_Item,Data_Items_5_Count,Data_Items_5_m_Item,Data_Items_6_Count,Data_Items_6_m_Item,Data_Items_7_Count,Data_Items_7_m_Item,Data_Items_8_Count,Data_Items_8_m_Item,Data_Items_9_Count,Data_Items_9_m_Item,Data_Setting,Data_Type,Data_m_Area,Filepath,Area_Name,Area_Name_Ru,Data_Items_0_m_Item_stringkey,Data_Items_1_m_Item_stringkey,Data_Items_2_m_Item_stringkey,Data_Items_3_m_Item_stringkey,Data_Items_4_m_Item_stringkey,Data_Items_5_m_Item_stringkey,Data_Items_6_m_Item_stringkey,Data_Items_7_m_Item_stringkey,Data_Items_8_m_Item_stringkey,Data_Items_9_m_Item_stringkey,Data_Items_0_m_Item_text,Data_Items_1_m_Item_text,Data_Items_2_m_Item_text,Data_Items_3_m_Item_text,Data_Items_4_m_Item_text,Data_Items_5_m_Item_text,Data_Items_6_m_Item_text,Data_Items_7_m_Item_text,Data_Items_8_m_Item_text,Data_Items_9_m_Item_text,All_Items,All_Item_Ids
0,0,919118b9d72840dabef59fb9619a8b20,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,AmourCourtoisBook,False,1,2841782e80bc48b1927677b3a88e392e,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,66b9351d457f4363afebcbca96a506da,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_AmourCourtoisBook.jbp,Upper Decks,Верхние палубы,bfc73060-7be4-425c-a2c2-4f0ff2d287c2,,,,,,,,,,Secrets of Gallant Conduct: How to Express Your Feelings to a Noble Lady,,,,,,,,,,Secrets of Gallant Conduct: How to Express Your Feelings to a Noble Lady,2841782e80bc48b1927677b3a88e392e
1,1,136d2833bd834cdab8cb0e8b65ea1bdc,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,GoodLootHidden,False,,,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,54b22e52a6da46eaa8c0f1fe2049d040,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_GoodLootHidden.jbp,Ruined Palace,Разрушенный дворец,,,,,,,,,,,,,,,,,,,,,,
2,2,0fe886222a234ad8abd7022a2d6ef04f,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,Loot4,False,1,199b3d86b5214c0c8fe483806f3122ce,3,edf1ca959e0f40f4bf5bb64586f816b9,1,04292c838432460396cfce58af861338,1,e29803f0fb444c90a153a2563cf30ae7,1,e239a9c0eaa84444a12b5aff054a0085,,,,,,,,,,,AdeptusMechanicus,Good,5abaf24cde1f4674a658da370195f537,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_Loot4.jbp,Outer Spire Halls,Внешние палаты шпиля,ce572ed4-94db-4ebf-8762-ccbb5671c404,45090443-c462-4cc5-8bef-f7d269a0bbdc,6d2d536e-77ac-4fd6-87c9-a2eb9a8f5cd1,0895bcbb-c5b8-4049-b516-859215d8735c,4839b9f3-2ed5-4a16-b895-a6bf99c21a88,,,,,,Armour of Despair,Xenospasm Grenade,Avenger Tactical Gauntlets,Peripheral Control Monocle,Melta Charge,,,,,,"Armour of Despair, Xenospasm Grenade, Avenger Tactical Gauntlets, Peripheral Control Monocle, Melta Charge","199b3d86b5214c0c8fe483806f3122ce, edf1ca959e0f40f4bf5bb64586f816b9, 04292c838432460396cfce58af861338, e29803f0fb444c90a153a2563cf30ae7, e239a9c0eaa84444a12b5aff054a0085"
3,3,a613e4a00a66493a82eb9609ecb8cc61,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (3),False,1,f123c305357a44f58b1d6449ded7cda7,1,8d9ec005c04a4984ad6032b3f74d84b1,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_LootGood (3).jbp,Upper Decks,Верхние палубы,e9a5f200-a460-4f0d-ae4e-69a58468d2d7,ff264714-1839-493e-90f0-b48cd7c9976c,,,,,,,,,Master Surgeon Gloves,Advanced Medikit,,,,,,,,,"Master Surgeon Gloves, Advanced Medikit","f123c305357a44f58b1d6449ded7cda7, 8d9ec005c04a4984ad6032b3f74d84b1"
4,4,aed888571d4c4e69b5e063e36ddddfb5,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (5),False,1,60989b213e8547799e6c17ed9b0238c0,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\Adeptu

Short version of game loot:

In [50]:
goods_df_short = goods_df.query('All_Items!=""')[['AssetId', 'Data_ContainerName', 'Area_Name', 'Area_Name_Ru', 'All_Items','All_Item_Ids']]

display(goods_df_short.head(10))

,AssetId,Data_ContainerName,Area_Name,Area_Name_Ru,All_Items,All_Item_Ids
0,919118b9d72840dabef59fb9619a8b20,AmourCourtoisBook,Upper Decks,Верхние палубы,Secrets of Gallant Conduct: How to Express Your Feelings to a Noble Lady,2841782e80bc48b1927677b3a88e392e
2,0fe886222a234ad8abd7022a2d6ef04f,Loot4,Outer Spire Halls,Внешние палаты шпиля,"Armour of Despair, Xenospasm Grenade, Avenger Tactical Gauntlets, Peripheral Control Monocle, Melta Charge","199b3d86b5214c0c8fe483806f3122ce, edf1ca959e0f40f4bf5bb64586f816b9, 04292c838432460396cfce58af861338, e29803f0fb444c90a153a2563cf30ae7, e239a9c0eaa84444a12b5aff054a0085"
3,a613e4a00a66493a82eb9609ecb8cc61,LootGood (3),Upper Decks,Верхние палубы,"Master Surgeon Gloves, Advanced Medikit","f123c305357a44f58b1d6449ded7cda7, 8d9ec005c04a4984ad6032b3f74d84b1"
4,aed888571d4c4e69b5e063e36ddddfb5,LootGood (5),Upper Decks,Верхние палубы,Arch-Strategist's Visor,60989b213e8547799e6c17ed9b0238c0
5,b571c287b06b4b2a95376ccd0dc33875,LootGood,Upper Decks,Верхние палубы,"Possessed Daemon Engine Heart, [Mars-Pattern] Plasma Gun","f08a955b6c154086bd3c1b1445ef211a, e53837d23baa40119092c9c185ee9a6b, ece61084f54a452cb63c135dab7c2b0f"
6,5cf71651f7274af6bc7cf5ddc4846e29,LootGood1,Ark Mechanicus Hermetico,"Корабль-ковчег ""Герметико""","Heavy Lifters, Mark 8B3-5 Filter Gas Mask","a2639877686f48f3b659f4c6b95709d8, 7bbee3ac45c54dccb8563a2c75941c44"
7,98fb511d36344c6d88cc2be7b01b182e,Loot_Good (1),Ruined Palace,Разрушенный дворец,"Toxin Grenade, Antitoxin, Immobilising Grenade, Battle Stimulator Medikit","705dcc9dda9d429e9de2c8f974e632b6, 02e3fe7902f04c2f8bac7c9b15abd8ae, e397cb9f6a1140a3bd9108aa43dc2b16, d878a044b9174fc7b663ba94b0aee377"
8,4a3e8f0715564e61b96a761a9f31e867,Loot_Good (3),Ruined Palace,Разрушенный дворец,"Crusade Plate Gloves, Shroud of Dauntlessness, Enforcer Light Boots","f939d14ed48549ab8c3de39a6ef0e676, c6ee4e6fec43459f88c9576c9805bdb4, 5dbc1cb871f948c99dd27112e0e4a3fc"
9,fbe81dd6ec124be9a424aaf309ae27fd,necron_container_GoodLoot,Epitaph,Эпитафия,"Singer of Fearsome Sagas, Mind Assemble Circlet, Rallying Boots","5e536199ce074052babb98226bbd29fc, 4f654e32d91c4662b5337ae7a0555a9f, 7aad46fb5b8642c8b5b3ec962ed36c1c"
10,df2549f205a849e586917282c66206f0,PrologueOD_TechRoomChest01,Upper Decks,Верхние палубы,"Machine Rite Set, Lexmechanic's Goggles","0895828ef19a4245aaeca3335dd933de, 1b0138241800480183dd871a42eabe7d"


 Create for russian version:

In [51]:
#make separate copy of goods_df for russian version to prevent unwanted changes in goods_df
goods_df_ru = goods_df.copy(deep= True)
#find item columns with names like Data_Items_{number}_m_Item
columns = find_item_columns(goods_df_ru)
goods_df_ru, columns = get_text_info(df = goods_df_ru,
                            map_df = assets_df[['AssetId','File']],
                            index_df = index_df_ru,
                            columns_in = columns)

In [52]:
display(goods_df.head())

,index,AssetId,Data_$type,Data_CargoVolumeAmount,Data_ContainerName,Data_Identify,Data_Items_0_Count,Data_Items_0_m_Item,Data_Items_1_Count,Data_Items_1_m_Item,Data_Items_2_Count,Data_Items_2_m_Item,Data_Items_3_Count,Data_Items_3_m_Item,Data_Items_4_Count,Data_Items_4_m_Item,Data_Items_5_Count,Data_Items_5_m_Item,Data_Items_6_Count,Data_Items_6_m_Item,Data_Items_7_Count,Data_Items_7_m_Item,Data_Items_8_Count,Data_Items_8_m_Item,Data_Items_9_Count,Data_Items_9_m_Item,Data_Setting,Data_Type,Data_m_Area,Filepath,Area_Name,Area_Name_Ru,Data_Items_0_m_Item_stringkey,Data_Items_1_m_Item_stringkey,Data_Items_2_m_Item_stringkey,Data_Items_3_m_Item_stringkey,Data_Items_4_m_Item_stringkey,Data_Items_5_m_Item_stringkey,Data_Items_6_m_Item_stringkey,Data_Items_7_m_Item_stringkey,Data_Items_8_m_Item_stringkey,Data_Items_9_m_Item_stringkey,Data_Items_0_m_Item_text,Data_Items_1_m_Item_text,Data_Items_2_m_Item_text,Data_Items_3_m_Item_text,Data_Items_4_m_Item_text,Data_Items_5_m_Item_text,Data_Items_6_m_Item_text,Data_Items_7_m_Item_text,Data_Items_8_m_Item_text,Data_Items_9_m_Item_text,All_Items,All_Item_Ids
0,0,919118b9d72840dabef59fb9619a8b20,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,AmourCourtoisBook,False,1,2841782e80bc48b1927677b3a88e392e,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,66b9351d457f4363afebcbca96a506da,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_AmourCourtoisBook.jbp,Upper Decks,Верхние палубы,bfc73060-7be4-425c-a2c2-4f0ff2d287c2,,,,,,,,,,Secrets of Gallant Conduct: How to Express Your Feelings to a Noble Lady,,,,,,,,,,Secrets of Gallant Conduct: How to Express Your Feelings to a Noble Lady,2841782e80bc48b1927677b3a88e392e
1,1,136d2833bd834cdab8cb0e8b65ea1bdc,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,GoodLootHidden,False,,,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,54b22e52a6da46eaa8c0f1fe2049d040,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_GoodLootHidden.jbp,Ruined Palace,Разрушенный дворец,,,,,,,,,,,,,,,,,,,,,,
2,2,0fe886222a234ad8abd7022a2d6ef04f,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,Loot4,False,1,199b3d86b5214c0c8fe483806f3122ce,3,edf1ca959e0f40f4bf5bb64586f816b9,1,04292c838432460396cfce58af861338,1,e29803f0fb444c90a153a2563cf30ae7,1,e239a9c0eaa84444a12b5aff054a0085,,,,,,,,,,,AdeptusMechanicus,Good,5abaf24cde1f4674a658da370195f537,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_Loot4.jbp,Outer Spire Halls,Внешние палаты шпиля,ce572ed4-94db-4ebf-8762-ccbb5671c404,45090443-c462-4cc5-8bef-f7d269a0bbdc,6d2d536e-77ac-4fd6-87c9-a2eb9a8f5cd1,0895bcbb-c5b8-4049-b516-859215d8735c,4839b9f3-2ed5-4a16-b895-a6bf99c21a88,,,,,,Armour of Despair,Xenospasm Grenade,Avenger Tactical Gauntlets,Peripheral Control Monocle,Melta Charge,,,,,,"Armour of Despair, Xenospasm Grenade, Avenger Tactical Gauntlets, Peripheral Control Monocle, Melta Charge","199b3d86b5214c0c8fe483806f3122ce, edf1ca959e0f40f4bf5bb64586f816b9, 04292c838432460396cfce58af861338, e29803f0fb444c90a153a2563cf30ae7, e239a9c0eaa84444a12b5aff054a0085"
3,3,a613e4a00a66493a82eb9609ecb8cc61,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (3),False,1,f123c305357a44f58b1d6449ded7cda7,1,8d9ec005c04a4984ad6032b3f74d84b1,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_LootGood (3).jbp,Upper Decks,Верхние палубы,e9a5f200-a460-4f0d-ae4e-69a58468d2d7,ff264714-1839-493e-90f0-b48cd7c9976c,,,,,,,,,Master Surgeon Gloves,Advanced Medikit,,,,,,,,,"Master Surgeon Gloves, Advanced Medikit","f123c305357a44f58b1d6449ded7cda7, 8d9ec005c04a4984ad6032b3f74d84b1"
4,4,aed888571d4c4e69b5e063e36ddddfb5,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (5),False,1,60989b213e8547799e6c17ed9b0238c0,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\Adeptu

In [53]:
#Create aggregate column of contained items in area
goods_df_ru['All_Items'] = goods_df_ru[columns].astype(str).agg(lambda x: ', '.join(word for word in x if word.strip()), axis = 1)
item_cols = find_item_columns(goods_df)
goods_df['All_Item_Ids'] = goods_df[item_cols].astype(str).agg(lambda x: ', '.join(word for word in x if word.strip()), axis = 1)
goods_df_ru.head()

,index,AssetId,Data_$type,Data_CargoVolumeAmount,Data_ContainerName,Data_Identify,Data_Items_0_Count,Data_Items_0_m_Item,Data_Items_1_Count,Data_Items_1_m_Item,Data_Items_2_Count,Data_Items_2_m_Item,Data_Items_3_Count,Data_Items_3_m_Item,Data_Items_4_Count,Data_Items_4_m_Item,Data_Items_5_Count,Data_Items_5_m_Item,Data_Items_6_Count,Data_Items_6_m_Item,Data_Items_7_Count,Data_Items_7_m_Item,Data_Items_8_Count,Data_Items_8_m_Item,Data_Items_9_Count,Data_Items_9_m_Item,Data_Setting,Data_Type,Data_m_Area,Filepath,Area_Name,Area_Name_Ru,Data_Items_0_m_Item_stringkey,Data_Items_1_m_Item_stringkey,Data_Items_2_m_Item_stringkey,Data_Items_3_m_Item_stringkey,Data_Items_4_m_Item_stringkey,Data_Items_5_m_Item_stringkey,Data_Items_6_m_Item_stringkey,Data_Items_7_m_Item_stringkey,Data_Items_8_m_Item_stringkey,Data_Items_9_m_Item_stringkey,Data_Items_0_m_Item_text,Data_Items_1_m_Item_text,Data_Items_2_m_Item_text,Data_Items_3_m_Item_text,Data_Items_4_m_Item_text,Data_Items_5_m_Item_text,Data_Items_6_m_Item_text,Data_Items_7_m_Item_text,Data_Items_8_m_Item_text,Data_Items_9_m_Item_text,All_Items,All_Item_Ids
0,0,919118b9d72840dabef59fb9619a8b20,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,AmourCourtoisBook,False,1,2841782e80bc48b1927677b3a88e392e,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,66b9351d457f4363afebcbca96a506da,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_AmourCourtoisBook.jbp,Upper Decks,Верхние палубы,bfc73060-7be4-425c-a2c2-4f0ff2d287c2,,,,,,,,,,Секреты галантного поведения: как выразить чувства перед благородной леди,,,,,,,,,,Секреты галантного поведения: как выразить чувства перед благородной леди,2841782e80bc48b1927677b3a88e392e
1,1,136d2833bd834cdab8cb0e8b65ea1bdc,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,GoodLootHidden,False,,,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,54b22e52a6da46eaa8c0f1fe2049d040,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_GoodLootHidden.jbp,Ruined Palace,Разрушенный дворец,,,,,,,,,,,,,,,,,,,,,,
2,2,0fe886222a234ad8abd7022a2d6ef04f,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,Loot4,False,1,199b3d86b5214c0c8fe483806f3122ce,3,edf1ca959e0f40f4bf5bb64586f816b9,1,04292c838432460396cfce58af861338,1,e29803f0fb444c90a153a2563cf30ae7,1,e239a9c0eaa84444a12b5aff054a0085,,,,,,,,,,,AdeptusMechanicus,Good,5abaf24cde1f4674a658da370195f537,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_Loot4.jbp,Outer Spire Halls,Внешние палаты шпиля,ce572ed4-94db-4ebf-8762-ccbb5671c404,45090443-c462-4cc5-8bef-f7d269a0bbdc,6d2d536e-77ac-4fd6-87c9-a2eb9a8f5cd1,0895bcbb-c5b8-4049-b516-859215d8735c,4839b9f3-2ed5-4a16-b895-a6bf99c21a88,,,,,,Броня отчаяния,Ксеноспазм-граната,Тактические наручи мстителя,Монокль периферийного контроля,Мельта-заряд,,,,,,"Броня отчаяния, Ксеноспазм-граната, Тактические наручи мстителя, Монокль периферийного контроля, Мельта-заряд","199b3d86b5214c0c8fe483806f3122ce, edf1ca959e0f40f4bf5bb64586f816b9, 04292c838432460396cfce58af861338, e29803f0fb444c90a153a2563cf30ae7, e239a9c0eaa84444a12b5aff054a0085"
3,3,a613e4a00a66493a82eb9609ecb8cc61,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (3),False,1,f123c305357a44f58b1d6449ded7cda7,1,8d9ec005c04a4984ad6032b3f74d84b1,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_LootGood (3).jbp,Upper Decks,Верхние палубы,e9a5f200-a460-4f0d-ae4e-69a58468d2d7,ff264714-1839-493e-90f0-b48cd7c9976c,,,,,,,,,Перчатки опытного хирурга,Улучшенный медкомплект,,,,,,,,,"Перчатки опытного хирурга, Улучшенный медкомплект","f123c305357a44f58b1d6449ded7cda7, 8d9ec005c04a4984ad6032b3f74d84b1"
4,4,aed888571d4c4e69b5e063e36ddddfb5,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (5),False,1,60989b213e8547799e6c17ed9b0238c0,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blu

In [54]:
goods_df_short_ru = goods_df_ru.query('All_Items!=""')[['AssetId', 'Data_ContainerName', 'Area_Name', 'Area_Name_Ru', 'All_Items','All_Item_Ids']]

display(goods_df_short_ru.head(10))

,AssetId,Data_ContainerName,Area_Name,Area_Name_Ru,All_Items,All_Item_Ids
0,919118b9d72840dabef59fb9619a8b20,AmourCourtoisBook,Upper Decks,Верхние палубы,Секреты галантного поведения: как выразить чувства перед благородной леди,2841782e80bc48b1927677b3a88e392e
2,0fe886222a234ad8abd7022a2d6ef04f,Loot4,Outer Spire Halls,Внешние палаты шпиля,"Броня отчаяния, Ксеноспазм-граната, Тактические наручи мстителя, Монокль периферийного контроля, Мельта-заряд","199b3d86b5214c0c8fe483806f3122ce, edf1ca959e0f40f4bf5bb64586f816b9, 04292c838432460396cfce58af861338, e29803f0fb444c90a153a2563cf30ae7, e239a9c0eaa84444a12b5aff054a0085"
3,a613e4a00a66493a82eb9609ecb8cc61,LootGood (3),Upper Decks,Верхние палубы,"Перчатки опытного хирурга, Улучшенный медкомплект","f123c305357a44f58b1d6449ded7cda7, 8d9ec005c04a4984ad6032b3f74d84b1"
4,aed888571d4c4e69b5e063e36ddddfb5,LootGood (5),Upper Decks,Верхние палубы,Визор архистратега,60989b213e8547799e6c17ed9b0238c0
5,b571c287b06b4b2a95376ccd0dc33875,LootGood,Upper Decks,Верхние палубы,"Сердце одержимого демонического орудия, [Модель Марс] Плазма-ружье","f08a955b6c154086bd3c1b1445ef211a, e53837d23baa40119092c9c185ee9a6b, ece61084f54a452cb63c135dab7c2b0f"
6,5cf71651f7274af6bc7cf5ddc4846e29,LootGood1,Ark Mechanicus Hermetico,"Корабль-ковчег ""Герметико""","Перчатки для подъема тяжестей, Противогаз ""Марк 8B3-5""","a2639877686f48f3b659f4c6b95709d8, 7bbee3ac45c54dccb8563a2c75941c44"
7,98fb511d36344c6d88cc2be7b01b182e,Loot_Good (1),Ruined Palace,Разрушенный дворец,"Токсичная граната, Антитоксин, Обездвиживающая граната, Медкомплект с боевыми стимуляторами","705dcc9dda9d429e9de2c8f974e632b6, 02e3fe7902f04c2f8bac7c9b15abd8ae, e397cb9f6a1140a3bd9108aa43dc2b16, d878a044b9174fc7b663ba94b0aee377"
8,4a3e8f0715564e61b96a761a9f31e867,Loot_Good (3),Ruined Palace,Разрушенный дворец,"Латные перчатки крестового похода, Покров неустрашимости, Легкие сапоги бойца","f939d14ed48549ab8c3de39a6ef0e676, c6ee4e6fec43459f88c9576c9805bdb4, 5dbc1cb871f948c99dd27112e0e4a3fc"
9,fbe81dd6ec124be9a424aaf309ae27fd,necron_container_GoodLoot,Epitaph,Эпитафия,"Певец свирепых вис, Венец собранности, Стимулирующие сапоги","5e536199ce074052babb98226bbd29fc, 4f654e32d91c4662b5337ae7a0555a9f, 7aad46fb5b8642c8b5b3ec962ed36c1c"
10,df2549f205a849e586917282c66206f0,PrologueOD_TechRoomChest01,Upper Decks,Верхние палубы,"Набор для машинного ритуала, Очки лексмеханика","0895828ef19a4245aaeca3335dd933de, 1b0138241800480183dd871a42eabe7d"


#### Create dataframe of item ids and locations

In [55]:
item_areas_df = create_df_item_location(goods_df)
display(item_areas_df)

,Item_Id,Area_Name,Area_Name_Ru
0,2841782e80bc48b1927677b3a88e392e,Upper Decks,Верхние палубы
1,199b3d86b5214c0c8fe483806f3122ce,Outer Spire Halls,Внешние палаты шпиля
2,f123c305357a44f58b1d6449ded7cda7,Upper Decks,Верхние палубы
3,60989b213e8547799e6c17ed9b0238c0,Upper Decks,Верхние палубы
4,f08a955b6c154086bd3c1b1445ef211a,Upper Decks,Верхние палубы
...,...,...,...
417,a5130a7096a546bf883574db563c4c57,Chasm,Прорва
418,cd2541b15c4249fd974c3a2a0695aa70,Chasm,Прорва
419,053074cada484310a920573477ebc7ed,Chasm,Прорва
420,a676f7b14cf445159680c73184e2aaea,Chasm,Прорва


### Vendor table

#### Folders

In [56]:
folder_vendor_factions = r'e:\Steam\RogueTrader\WorkFolder\Blueprints\VendorFactions'
folder_vendor_tables = r'e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables'

#### Making table

In [57]:
list_of_vendor_files = []
for root, dirs, files in os.walk(folder_vendor_tables):        
        for file in files:           
                list_of_vendor_files.append(os.path.join(root, file))
print('The number of files in the folder: ', len(list_of_vendor_files))

The number of files in the folder:  19


Needed columns:
```"Data": {
    "$type": "ccc43623dd9341449b5d07be1dabaa23, BlueprintSharedVendorTable",
    "PrototypeLink": "",
    "m_Overrides": [],
    "Components": [
      {
        "$type": "9a9cba603f85c634690eb67962fdf792, LootItemsPackFixed",
        "name": "$LootItemsPackFixed$f44f5cc2-14dc-489b-a625-52dcbb8a11c4",
        "m_Flags": 0,
        "PrototypeLink": {
          "guid": "",
          "name": ""
        },
        "m_Overrides": [],
        "m_OverrideReputationPointsToUnlock": true,
        "m_ReputationPointsToUnlock": 66105,
        "m_Item": {
          "m_Type": "Item",
          "m_Item": "!bp_9637dc3883074c6ca17bb0ba6bee8605",
          "m_Loot": null,
          "m_OverrideProfitFactorCost": true,
          "m_ProfitFactorCost": 57
        },
        "m_Count": 6
      },```

Example of making dataframe from single file:

In [58]:
vendor_items = []
file = r'DrusianChapter4_VendorTable_Base.jbp'
filepath = '{}\\{}'.format(folder_vendor_tables,file)
with open(filepath) as asset_file:           
                file_contents = asset_file.read()
                data = json.loads(file_contents)
for item in data['Data']['Components']:
    #print(item)
    file_info = {}
    file_info['AssetId'] = data['AssetId']
    file_info['Item_Id'] = clear_rows(item['m_Item']['m_Item'])
    file_info['ProfitFactorCost'] = item['m_Item']['m_ProfitFactorCost']
    file_info['RepToUnlock'] = item['m_ReputationPointsToUnlock']
    file_info['Count'] = item['m_Count']
    file_info['Filepath'] = filepath    
    vendor_items.append(file_info)



Creation of table of items:

In [59]:
vendor_items_df = pd.DataFrame()
for file in list_of_vendor_files:
    with open(file) as asset_file:                   
        file_contents = asset_file.read()
        data = json.loads(file_contents)
        vendor_items = []
        for item in data['Data']['Components']:    
            file_info = {}
            file_info['AssetId'] = data['AssetId']
            file_info['Item_Id'] = clear_rows(item['m_Item']['m_Item'])
            file_info['ProfitFactorCost'] = item['m_Item']['m_ProfitFactorCost']
            file_info['RepToUnlock'] = item['m_ReputationPointsToUnlock']
            file_info['Count'] = item['m_Count']
            file_info['Filepath'] = file    
            vendor_items.append(file_info)
        temp_df =pd.DataFrame.from_dict(vendor_items)
        vendor_items_df = pd.concat([vendor_items_df,temp_df], axis = 0) 



In [60]:
vendor_items_df = vendor_items_df.fillna('')

#### Add column with faction names

In [61]:
# function to get filenames from filepathes
def get_faction(filename):
    head, tail = os.path.split(filename)
    temp = tail.split('_')
    return temp[0]
vendor_items_df = vendor_items_df.assign(Vendor_Name = lambda row: row['Filepath'].apply(get_faction))

#### Add columns with item names

In [62]:
def find_stringkeys(row):
        """Function to get strinkey from one of two dict keys responsible for them"""    
        try:
            row = row.strip()
            with open(row) as asset_file:           
                file_contents = asset_file.read()
                data = json.loads(file_contents)
                try:
                    if data['Data']['m_DisplayName']['Shared'] is not None:
                        key_list = [data['Data']['m_DisplayName']['m_Key'], data['Data']['m_DisplayName']['Shared']['stringkey']]
                        for key in key_list:                     
                            a = match_pattern(key) 
                            if a:                 
                                return key
                    else:
                        key = data['Data']['m_DisplayName']['m_Key']
                        return key
                except Exception as err:
                    print(f'Unexpected {err=}, {type(err)=}',row)
                #print(data['Data']['m_DisplayName']['Shared'])
                
        except:
            return ''    

In [63]:
vendor_items_df['Item_Id'] = vendor_items_df['Item_Id'].str.strip()         
assets_dict = dict(zip(assets_df.iloc[:,0],assets_df.iloc[:,1]))
filenames_series = vendor_items_df['Item_Id'].map(assets_dict)
vendor_items_df['{}_stringkey'.format('Item_Id')]  = filenames_series.apply(find_stringkeys)

In [64]:
map_dict = dict(zip(index_df.iloc[:,0],index_df.iloc[:,1]))
vendor_items_df['Item_Name'] = vendor_items_df['Item_Id_stringkey'].map(map_dict)
#russian version
map_dict_ru = dict(zip(index_df_ru.iloc[:,0],index_df_ru.iloc[:,1]))
vendor_items_df['Item_Name_Ru'] = vendor_items_df['Item_Id_stringkey'].map(map_dict_ru)
display(vendor_items_df.shape[0])
display(vendor_items_df.head())

330

,AssetId,Item_Id,ProfitFactorCost,RepToUnlock,Count,Filepath,Vendor_Name,Item_Id_stringkey,Item_Name,Item_Name_Ru
0,c330647e71a547309301a16b1904830b,56e3576b84384495b9015d5ec88217ac,20,9560,5,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Advanced.jbp,DrusianChapter2,0e987eda-d900-4c72-bd08-59d86c67643c,Medikit,Медкомплект
1,c330647e71a547309301a16b1904830b,56e3576b84384495b9015d5ec88217ac,35,15810,5,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Advanced.jbp,DrusianChapter2,0e987eda-d900-4c72-bd08-59d86c67643c,Medikit,Медкомплект
2,c330647e71a547309301a16b1904830b,b6daa597bb6b5114bb9e21ea1e855792,26,25230,5,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Advanced.jbp,DrusianChapter2,5f33b743-e2b7-4353-9f5b-29f6e4cd71ba,Onslaught,Резня
3,c330647e71a547309301a16b1904830b,20650117b63acf740998b602e108c51a,32,28310,7,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Advanced.jbp,DrusianChapter2,f54981ae-fa49-45b5-a97e-5c7695f74449,Krak Grenade,Крак-граната
4,c330647e71a547309301a16b1904830b,b6daa597bb6b5114bb9e21ea1e855792,29,37730,5,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Advanced.jbp,DrusianChapter2,5f33b743-e2b7-4353-9f5b-29f6e4cd71ba,Onslaught,Резня


**Notes**:  
```Liege``` - Kasballica Mission. 

In [65]:
vendor_items_df[vendor_items_df['Item_Name'].isna()]

,AssetId,Item_Id,ProfitFactorCost,RepToUnlock,Count,Filepath,Vendor_Name,Item_Id_stringkey,Item_Name,Item_Name_Ru
6,f8a16dab0d494db4a936b7c54da21576,266766be7aab4142af87ba3e76e0ddce,23,9560,1,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Base.jbp,DrusianChapter2,,NaN,NaN
6,9d38eb7f5531476cbe0a9f3e32e010ad,80de65062c0043efb920037ed90c56ae,69,77730,1,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter4_VendorTable_Base.jbp,DrusianChapter4,,NaN,NaN
11,9d38eb7f5531476cbe0a9f3e32e010ad,6a890b2546f94821be5dba77b85d4e98,79,85480,1,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter4_VendorTable_Base.jbp,DrusianChapter4,,NaN,NaN
20,9d38eb7f5531476cbe0a9f3e32e010ad,d891b6e2ca284b32aa5cfeafb07bdd29,97,107605,1,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter4_VendorTable_Base.jbp,DrusianChapter4,,NaN,NaN
4,d4cd37f07db24594931627eb1e41d574,81217d924b524919a959a3d3a02021b3,51,32680,5,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\LiegeChapter2_VendorTable_Advanced.jbp,LiegeChapter2,7c042a15-8b75-4070-8c44-98cbe2abc604,NaN,NaN
9,1b2c9e5ef38d45b4bcfe36eecd49f3a1,ece61084f54a452cb63c135dab7c2b0f,90,71575,3,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\LiegeChapter4_VendorTable_Base.jbp,LiegeChapter4,3c00a005-c74d-49ba-b23d-4de3eb68db60,NaN,NaN


Replacing substring 'Liege' with the substring 'KasballicaMission' in ```Vendor_Name```:

In [66]:
vendor_items_df =vendor_items_df.replace('Liege','KasballicaMission', regex=True)

#### Add vendor names in russian language

In [67]:
vendor_items_df['Vendor_Name'].unique()

array(['DrusianChapter2', 'DrusianChapter4', 'ExploratorChapter1',
       'ExploratorChapter2', 'ExploratorChapter4', 'HaemunculiChapter3',
       'KasballicaMissionChapter2', 'KasballicaMissionChapter4',
       'PirateChapter1', 'PirateChapter2', 'PirateChapter4',
       'PitBossChapter3', 'ShipVendor'], dtype=object)

In [68]:
en_names = vendor_items_df['Vendor_Name'].unique().tolist()
ru_names = ['Друзианцы_Глава_2', 'Друзианцы_Глава_4', 'Эксплораторы_Глава_1', 'Эксплораторы_Глава_2', 'Эксплораторы_Глава_4',
            'Тервантьяс_Глава_3', 'МиссияКасбаллики_Глава_2', 'МиссияКасбаллики_Глава_4', 'ПустотноеБратство_Глава_1',
            'ПустотноеБратство_Глава_2', 'ПустотноеБратство_Глава_4', 'Яма_Глава_3', 'ИмперскийФлот']
map_dict = dict(zip(en_names,ru_names))
vendor_items_df = vendor_items_df.assign(Vendor_Name_Ru = lambda row:row['Vendor_Name'].map(map_dict))

In [69]:
#filling nans with empty rows
vendor_items_df = vendor_items_df.fillna('')

In [70]:
#show final result
display(vendor_items_df.head())

,AssetId,Item_Id,ProfitFactorCost,RepToUnlock,Count,Filepath,Vendor_Name,Item_Id_stringkey,Item_Name,Item_Name_Ru,Vendor_Name_Ru
0,c330647e71a547309301a16b1904830b,56e3576b84384495b9015d5ec88217ac,20,9560,5,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Advanced.jbp,DrusianChapter2,0e987eda-d900-4c72-bd08-59d86c67643c,Medikit,Медкомплект,Друзианцы_Глава_2
1,c330647e71a547309301a16b1904830b,56e3576b84384495b9015d5ec88217ac,35,15810,5,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Advanced.jbp,DrusianChapter2,0e987eda-d900-4c72-bd08-59d86c67643c,Medikit,Медкомплект,Друзианцы_Глава_2
2,c330647e71a547309301a16b1904830b,b6daa597bb6b5114bb9e21ea1e855792,26,25230,5,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Advanced.jbp,DrusianChapter2,5f33b743-e2b7-4353-9f5b-29f6e4cd71ba,Onslaught,Резня,Друзианцы_Глава_2
3,c330647e71a547309301a16b1904830b,20650117b63acf740998b602e108c51a,32,28310,7,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Advanced.jbp,DrusianChapter2,f54981ae-fa49-45b5-a97e-5c7695f74449,Krak Grenade,Крак-граната,Друзианцы_Глава_2
4,c330647e71a547309301a16b1904830b,b6daa597bb6b5114bb9e21ea1e855792,29,37730,5,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Advanced.jbp,DrusianChapter2,5f33b743-e2b7-4353-9f5b-29f6e4cd71ba,Onslaught,Резня,Друзианцы_Глава_2


#### Final results

In [71]:
display(goods_df.head())


,index,AssetId,Data_$type,Data_CargoVolumeAmount,Data_ContainerName,Data_Identify,Data_Items_0_Count,Data_Items_0_m_Item,Data_Items_1_Count,Data_Items_1_m_Item,Data_Items_2_Count,Data_Items_2_m_Item,Data_Items_3_Count,Data_Items_3_m_Item,Data_Items_4_Count,Data_Items_4_m_Item,Data_Items_5_Count,Data_Items_5_m_Item,Data_Items_6_Count,Data_Items_6_m_Item,Data_Items_7_Count,Data_Items_7_m_Item,Data_Items_8_Count,Data_Items_8_m_Item,Data_Items_9_Count,Data_Items_9_m_Item,Data_Setting,Data_Type,Data_m_Area,Filepath,Area_Name,Area_Name_Ru,Data_Items_0_m_Item_stringkey,Data_Items_1_m_Item_stringkey,Data_Items_2_m_Item_stringkey,Data_Items_3_m_Item_stringkey,Data_Items_4_m_Item_stringkey,Data_Items_5_m_Item_stringkey,Data_Items_6_m_Item_stringkey,Data_Items_7_m_Item_stringkey,Data_Items_8_m_Item_stringkey,Data_Items_9_m_Item_stringkey,Data_Items_0_m_Item_text,Data_Items_1_m_Item_text,Data_Items_2_m_Item_text,Data_Items_3_m_Item_text,Data_Items_4_m_Item_text,Data_Items_5_m_Item_text,Data_Items_6_m_Item_text,Data_Items_7_m_Item_text,Data_Items_8_m_Item_text,Data_Items_9_m_Item_text,All_Items,All_Item_Ids
0,0,919118b9d72840dabef59fb9619a8b20,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,AmourCourtoisBook,False,1,2841782e80bc48b1927677b3a88e392e,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,66b9351d457f4363afebcbca96a506da,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_AmourCourtoisBook.jbp,Upper Decks,Верхние палубы,bfc73060-7be4-425c-a2c2-4f0ff2d287c2,,,,,,,,,,Secrets of Gallant Conduct: How to Express Your Feelings to a Noble Lady,,,,,,,,,,Secrets of Gallant Conduct: How to Express Your Feelings to a Noble Lady,2841782e80bc48b1927677b3a88e392e
1,1,136d2833bd834cdab8cb0e8b65ea1bdc,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,GoodLootHidden,False,,,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,54b22e52a6da46eaa8c0f1fe2049d040,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_GoodLootHidden.jbp,Ruined Palace,Разрушенный дворец,,,,,,,,,,,,,,,,,,,,,,
2,2,0fe886222a234ad8abd7022a2d6ef04f,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,Loot4,False,1,199b3d86b5214c0c8fe483806f3122ce,3,edf1ca959e0f40f4bf5bb64586f816b9,1,04292c838432460396cfce58af861338,1,e29803f0fb444c90a153a2563cf30ae7,1,e239a9c0eaa84444a12b5aff054a0085,,,,,,,,,,,AdeptusMechanicus,Good,5abaf24cde1f4674a658da370195f537,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_Loot4.jbp,Outer Spire Halls,Внешние палаты шпиля,ce572ed4-94db-4ebf-8762-ccbb5671c404,45090443-c462-4cc5-8bef-f7d269a0bbdc,6d2d536e-77ac-4fd6-87c9-a2eb9a8f5cd1,0895bcbb-c5b8-4049-b516-859215d8735c,4839b9f3-2ed5-4a16-b895-a6bf99c21a88,,,,,,Armour of Despair,Xenospasm Grenade,Avenger Tactical Gauntlets,Peripheral Control Monocle,Melta Charge,,,,,,"Armour of Despair, Xenospasm Grenade, Avenger Tactical Gauntlets, Peripheral Control Monocle, Melta Charge","199b3d86b5214c0c8fe483806f3122ce, edf1ca959e0f40f4bf5bb64586f816b9, 04292c838432460396cfce58af861338, e29803f0fb444c90a153a2563cf30ae7, e239a9c0eaa84444a12b5aff054a0085"
3,3,a613e4a00a66493a82eb9609ecb8cc61,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (3),False,1,f123c305357a44f58b1d6449ded7cda7,1,8d9ec005c04a4984ad6032b3f74d84b1,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_LootGood (3).jbp,Upper Decks,Верхние палубы,e9a5f200-a460-4f0d-ae4e-69a58468d2d7,ff264714-1839-493e-90f0-b48cd7c9976c,,,,,,,,,Master Surgeon Gloves,Advanced Medikit,,,,,,,,,"Master Surgeon Gloves, Advanced Medikit","f123c305357a44f58b1d6449ded7cda7, 8d9ec005c04a4984ad6032b3f74d84b1"
4,4,aed888571d4c4e69b5e063e36ddddfb5,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (5),False,1,60989b213e8547799e6c17ed9b0238c0,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\Adeptu

In [72]:
display(goods_df_ru.head())

,index,AssetId,Data_$type,Data_CargoVolumeAmount,Data_ContainerName,Data_Identify,Data_Items_0_Count,Data_Items_0_m_Item,Data_Items_1_Count,Data_Items_1_m_Item,Data_Items_2_Count,Data_Items_2_m_Item,Data_Items_3_Count,Data_Items_3_m_Item,Data_Items_4_Count,Data_Items_4_m_Item,Data_Items_5_Count,Data_Items_5_m_Item,Data_Items_6_Count,Data_Items_6_m_Item,Data_Items_7_Count,Data_Items_7_m_Item,Data_Items_8_Count,Data_Items_8_m_Item,Data_Items_9_Count,Data_Items_9_m_Item,Data_Setting,Data_Type,Data_m_Area,Filepath,Area_Name,Area_Name_Ru,Data_Items_0_m_Item_stringkey,Data_Items_1_m_Item_stringkey,Data_Items_2_m_Item_stringkey,Data_Items_3_m_Item_stringkey,Data_Items_4_m_Item_stringkey,Data_Items_5_m_Item_stringkey,Data_Items_6_m_Item_stringkey,Data_Items_7_m_Item_stringkey,Data_Items_8_m_Item_stringkey,Data_Items_9_m_Item_stringkey,Data_Items_0_m_Item_text,Data_Items_1_m_Item_text,Data_Items_2_m_Item_text,Data_Items_3_m_Item_text,Data_Items_4_m_Item_text,Data_Items_5_m_Item_text,Data_Items_6_m_Item_text,Data_Items_7_m_Item_text,Data_Items_8_m_Item_text,Data_Items_9_m_Item_text,All_Items,All_Item_Ids
0,0,919118b9d72840dabef59fb9619a8b20,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,AmourCourtoisBook,False,1,2841782e80bc48b1927677b3a88e392e,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,66b9351d457f4363afebcbca96a506da,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_AmourCourtoisBook.jbp,Upper Decks,Верхние палубы,bfc73060-7be4-425c-a2c2-4f0ff2d287c2,,,,,,,,,,Секреты галантного поведения: как выразить чувства перед благородной леди,,,,,,,,,,Секреты галантного поведения: как выразить чувства перед благородной леди,2841782e80bc48b1927677b3a88e392e
1,1,136d2833bd834cdab8cb0e8b65ea1bdc,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,GoodLootHidden,False,,,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,54b22e52a6da46eaa8c0f1fe2049d040,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_GoodLootHidden.jbp,Ruined Palace,Разрушенный дворец,,,,,,,,,,,,,,,,,,,,,,
2,2,0fe886222a234ad8abd7022a2d6ef04f,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,Loot4,False,1,199b3d86b5214c0c8fe483806f3122ce,3,edf1ca959e0f40f4bf5bb64586f816b9,1,04292c838432460396cfce58af861338,1,e29803f0fb444c90a153a2563cf30ae7,1,e239a9c0eaa84444a12b5aff054a0085,,,,,,,,,,,AdeptusMechanicus,Good,5abaf24cde1f4674a658da370195f537,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_Loot4.jbp,Outer Spire Halls,Внешние палаты шпиля,ce572ed4-94db-4ebf-8762-ccbb5671c404,45090443-c462-4cc5-8bef-f7d269a0bbdc,6d2d536e-77ac-4fd6-87c9-a2eb9a8f5cd1,0895bcbb-c5b8-4049-b516-859215d8735c,4839b9f3-2ed5-4a16-b895-a6bf99c21a88,,,,,,Броня отчаяния,Ксеноспазм-граната,Тактические наручи мстителя,Монокль периферийного контроля,Мельта-заряд,,,,,,"Броня отчаяния, Ксеноспазм-граната, Тактические наручи мстителя, Монокль периферийного контроля, Мельта-заряд","199b3d86b5214c0c8fe483806f3122ce, edf1ca959e0f40f4bf5bb64586f816b9, 04292c838432460396cfce58af861338, e29803f0fb444c90a153a2563cf30ae7, e239a9c0eaa84444a12b5aff054a0085"
3,3,a613e4a00a66493a82eb9609ecb8cc61,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (3),False,1,f123c305357a44f58b1d6449ded7cda7,1,8d9ec005c04a4984ad6032b3f74d84b1,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_LootGood (3).jbp,Upper Decks,Верхние палубы,e9a5f200-a460-4f0d-ae4e-69a58468d2d7,ff264714-1839-493e-90f0-b48cd7c9976c,,,,,,,,,Перчатки опытного хирурга,Улучшенный медкомплект,,,,,,,,,"Перчатки опытного хирурга, Улучшенный медкомплект","f123c305357a44f58b1d6449ded7cda7, 8d9ec005c04a4984ad6032b3f74d84b1"
4,4,aed888571d4c4e69b5e063e36ddddfb5,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (5),False,1,60989b213e8547799e6c17ed9b0238c0,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blu

In [73]:
display(item_areas_df.head())

,Item_Id,Area_Name,Area_Name_Ru
0,2841782e80bc48b1927677b3a88e392e,Upper Decks,Верхние палубы
1,199b3d86b5214c0c8fe483806f3122ce,Outer Spire Halls,Внешние палаты шпиля
2,f123c305357a44f58b1d6449ded7cda7,Upper Decks,Верхние палубы
3,60989b213e8547799e6c17ed9b0238c0,Upper Decks,Верхние палубы
4,f08a955b6c154086bd3c1b1445ef211a,Upper Decks,Верхние палубы


item "knuckle guard", Footfall atrium, canalization.

In [83]:
goods_df[goods_df['All_Item_Ids']=="9f512940203c4d15b243b7a20d289415"]

,index,AssetId,Data_$type,Data_CargoVolumeAmount,Data_ContainerName,Data_Identify,Data_Items_0_Count,Data_Items_0_m_Item,Data_Items_1_Count,Data_Items_1_m_Item,Data_Items_2_Count,Data_Items_2_m_Item,Data_Items_3_Count,Data_Items_3_m_Item,Data_Items_4_Count,Data_Items_4_m_Item,Data_Items_5_Count,Data_Items_5_m_Item,Data_Items_6_Count,Data_Items_6_m_Item,Data_Items_7_Count,Data_Items_7_m_Item,Data_Items_8_Count,Data_Items_8_m_Item,Data_Items_9_Count,Data_Items_9_m_Item,Data_Setting,Data_Type,Data_m_Area,Filepath,Area_Name,Area_Name_Ru,Data_Items_0_m_Item_stringkey,Data_Items_1_m_Item_stringkey,Data_Items_2_m_Item_stringkey,Data_Items_3_m_Item_stringkey,Data_Items_4_m_Item_stringkey,Data_Items_5_m_Item_stringkey,Data_Items_6_m_Item_stringkey,Data_Items_7_m_Item_stringkey,Data_Items_8_m_Item_stringkey,Data_Items_9_m_Item_stringkey,Data_Items_0_m_Item_text,Data_Items_1_m_Item_text,Data_Items_2_m_Item_text,Data_Items_3_m_Item_text,Data_Items_4_m_Item_text,Data_Items_5_m_Item_text,Data_Items_6_m_Item_text,Data_Items_7_m_Item_text,Data_Items_8_m_Item_text,Data_Items_9_m_Item_text,All_Items,All_Item_Ids
137,138,4e8f976be54840419151807aa8fa6b18,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,Good1,False,1,9f512940203c4d15b243b7a20d289415,,,,,,,,,,,,,,,,,,,ImperiumPoor,Good,1a93512eef8d4df9aad4deb12fc3af35,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR17\Forest_Good1.jbp,Footfall Atrium,Атриум Футфола,af13d6f3-7589-46c1-b0fd-c5ff1cfdb906,,,,,,,,,,Knuckle Guard,,,,,,,,,,Knuckle Guard,9f512940203c4d15b243b7a20d289415


In [85]:
goods_df['Data_Setting'].unique()

array(['AdeptusMechanicus', 'Drukhari', 'ImperiumMilitary',
       'ImperiumRich', 'Chaos', 'ImperiumPoor'], dtype=object)

## Appendix

### Service commands

In [84]:
assets_df.query('AssetId=="1a93512eef8d4df9aad4deb12fc3af35"')['File']

23064    e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Areas\Chapter_2\FootfallAtrium\FootfallAtrium.jbp
Name: File, dtype: object

In [75]:
index_df.query('index=="2dba87d7-8413-4147-afc4-8e315c24e0fd"')

,index,text
9198,2dba87d7-8413-4147-afc4-8e315c24e0fd,Kasballica Mission


### Saving tables

#english version
vendor_items_df.to_csv(save_folder + r'\vendor_items.csv', index=False)
goods_df.to_csv(save_folder + r'\loot_table.csv', index=False)
index_df.to_csv(save_folder + r'\index_df.csv', index=False)
item_areas_df.to_csv(save_folder + r'\item_areas_df.csv', index=False)
# ru version
index_df_ru.to_csv(save_folder + r'\index_df_ru.csv', index=False)
assets_df.to_csv(save_folder + r'\assets_df.csv', index=False)
goods_df_ru.to_csv(save_folder + r'\loot_table_ru.csv', index=False)



_____________

## LAB

In [81]:
equip_table = pd.read_csv(load_folder + r'\equip_items_df.csv')



In [82]:
display(equip_table.query('Item_Name=="Knuckle Guard"'))

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr
453,9f512940203c4d15b243b7a20d289415,"54663362c56fb114c9dc709407dcfc82, BlueprintItemEquipmentRing",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",83f5f65d42be4bac8ab60ff48c858c0d,NaN,NaN,NaN,NaN,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",9d6644fc39ad438cb62c7a01fd366cf6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,372fb310-4939-4a84-8c74-545290d7bbea,NaN,af13d6f3-7589-46c1-b0fd-c5ff1cfdb906,NaN,NaN,Tech,Unique,NaN,NaN,10.0,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Rings\Unique\KnuckleGuard_Item.jbp,Knuckle Guard,This knuckle guard grants the wearer a +5 bonus to Weapon Skill. It also grants any equipped melee weapons +5% armour penetration and +2 damage.,NaN,NaN


In [76]:
weapons_table = pd.read_csv(load_folder + r'\weapons.csv')
display(weapons_table.head())

,AssetId,Item_Name,Item_Descr,Add_Descr,Family,Is_Heavy,Holding_Type,Origin,Rarity,Damage_Type,Dodge_Penetration,Rate_Of_Fire,Min_Base_Damage,Max_Base_Damage,Max_Ammo,Max_Distance,Range,Penetration,Recoil,Cargo_Volume_Percent,Damage_Stat_Bonus_Factor,Ability_1_Name,Ability_1_Descr,Ability1_AP_Cost,Ability1_Type,Ability_2_Name,Ability_2_Descr,Ability2_AP_Cost,Ability2_Type,Ability_3_Name,Ability_3_Descr,Ability3_AP_Cost,Ability3_Type,Ability_4_Name,Ability_4_Descr,Ability4_AP_Cost,Ability4_Type,Ability_5_Name,Ability_5_Descr,Ability5_AP_Cost,Ability5_Type
0,24db44debaed469386fad90088c20ee9,Ancient Force Sword,This sword deals an additional +psy rating damage.,NaN,Force,False,OneHanded,MeleeWeaponry,Pattern,Energy,0,0,14,20,0,1,Melee,30,0,15,One,Strike,Single attack with equipped melee weapon.,1,SingleShot,Cleave,Wide slashing strike that affects multiple enemies at once.,2,AOE,Psychic Force,"Special attack that counts as a major psychic power.\nIt deals an additional (2 × psy rating + WP bonus) damage, but may trigger psychic phenomena and perils of the warp.",2,Custom,NaN,NaN,0,NaN,NaN,NaN,0,NaN
1,cd5fa172af484872820ffbb0d713985b,[Mezoa-Pattern] Arc Rifle,"When you hit the target with an arc rifle, you also attack any enemies within 2 cells of the target.",NaN,Solid,False,TwoHanded,RangedWeaponry,Pattern,Shock,0,1,13,20,4,18,Ranged,30,10,10,One,Single Shot,Single shot with equipped ranged weapon.,2,SingleShot,Reload,Reloads equipped weapon.,3,Reload,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN
2,bf9d592fc8444f29a99e27486182afb6,Alpha's Arc Rifle,"On a successful hit with this weapon, the wielder also attacks all targets in a 4-cell radius around the target.",NaN,Exotic,False,TwoHanded,RangedWeaponry,Pattern,Shock,0,1,30,40,4,16,Ranged,10,10,10,One,Single Shot,Single shot with equipped ranged weapon.,2,SingleShot,Reload,Reloads equipped weapon.,3,Reload,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN
3,89922abe7a3b4a1d9359ba7d316114fc,Arc Rifle,"When the wielder hits a target with the arc rifle, they also attack all enemies within 2 cells of the target.",NaN,Exotic,False,TwoHanded,RangedWeaponry,Common,Shock,0,1,10,15,4,18,Ranged,10,10,10,One,Single Shot,Single shot with equipped ranged weapon.,2,SingleShot,Reload,Reloads equipped weapon.,3,Reload,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN
4,0b1a6f7920114dafb5ffc2bdea37bf8d,Astartes Bolter,NaN,NaN,Bolt,False,TwoHanded,RangedWeaponry,Common,Impact,0,3,19,30,15,24,Ranged,25,25,10,One,Single Shot,Single shot with equipped ranged weapon.,1,SingleShot,Burst Fire,Burst fire with equipped ranged weapon.,2,Burst,Reload,Reloads equipped weapon.,3,Reload,Reload,Reloads equipped weapon.,4,NaN,NaN,NaN,0,NaN


In [77]:
t = create_df_item_location(goods_df)
display(t.head())

,Item_Id,Area_Name,Area_Name_Ru
0,2841782e80bc48b1927677b3a88e392e,Upper Decks,Верхние палубы
1,199b3d86b5214c0c8fe483806f3122ce,Outer Spire Halls,Внешние палаты шпиля
2,f123c305357a44f58b1d6449ded7cda7,Upper Decks,Верхние палубы
3,60989b213e8547799e6c17ed9b0238c0,Upper Decks,Верхние палубы
4,f08a955b6c154086bd3c1b1445ef211a,Upper Decks,Верхние палубы


In [78]:
t_weapons = t[t['Item_Id'].isin(weapons_table['AssetId'])]
display(t['Item_Id'].value_counts())

e239a9c0eaa84444a12b5aff054a0085    21
a01bd2e8da3e4a78805e92cde04419a0    16
56e3576b84384495b9015d5ec88217ac    12
0895828ef19a4245aaeca3335dd933de     7
0088d2c1ea084c1428266b7ffdeb6ab1     6
                                    ..
c431fcd14b45453e8fea6b2b4186778d     1
001c1a9ce35d48249539010b054cc1eb     1
643705c5a14b42ce8bfad4aa17ff494d     1
49d53db2e8eb41ee9e985926fb30c8d6     1
85b84712fc30435ea73f3d45049f25f2     1
Name: Item_Id, Length: 312, dtype: int64